# Export to JSON pipeline

Load the data (previously created pandas dataframe exported as a json)

In [34]:
import pandas as pd

In [35]:
data = pd.read_csv('./WIP_EXPORT_2.csv')

In [36]:
data.head(5)

,Unnamed: 0,contcod,year_survey,DummyY,totpop,ventile,ventile_income,pop,pop_share,ppp_factor,ppp_2011,ppp_percentile,x
0,0,ALB,2012,1,2.829337,1.0,48940.0,0.141467,0.05,43.857745,1115.880438,32.603788,2.5
1,1,ALB,2012,1,2.829337,2.0,68642.0,0.141467,0.05,43.857745,1565.105538,42.932962,7.5
2,2,ALB,2012,1,2.829337,3.0,78991.0,0.141467,0.05,43.857745,1801.072981,47.121037,12.5
3,3,ALB,2012,1,2.829337,4.0,87698.0,0.141467,0.05,43.857745,1999.601199,50.162916,17.5
4,4,ALB,2012,1,2.829337,5.0,95185.0,0.141467,0.05,43.857745,2170.312209,52.479402,22.5


In [37]:
data = data[['contcod', 'x', 'ppp_percentile']]

In [38]:
data.head(3)

,contcod,x,ppp_percentile
0,ALB,2.5,32.603788
1,ALB,7.5,42.932962
2,ALB,12.5,47.121037


Round to 2 decimal places everything other than 1st column

In [39]:
data['ppp_percentile'] = round(data['ppp_percentile'], 2)

In [40]:
countries = data['contcod'].unique()

In [41]:
countries[:10]

array(['ALB', 'ARG', 'ARM', 'AUS', 'AUT', 'BEN', 'BGD', 'BGR', 'BOL',
       'BRA'], dtype=object)

In [42]:
data.head(2)

,contcod,x,ppp_percentile
0,ALB,2.5,32.60
1,ALB,7.5,42.93


Rename the columns for personal convenience:

In [43]:
data.columns = ['contcod', 'x', 'y']

In [44]:
data.head(2)

,contcod,x,y
0,ALB,2.5,32.60
1,ALB,7.5,42.93


Create custom `dict` for export that looks like this:


```
{
   'IND': 
       [
           {'x': 2.5, 'y': 25.3}, 
           {'x': 7.5, 'y': 31.5}, 
           ... 
       ],
   'DEU': [  ...  ],
   ...
}
```

In [45]:
dict_for_export = {}

In [46]:
for country in countries:
  
  temp_list = []
  
  def append_to_list(x, y):
    temp_list.append({'x': x, 'y': y})
  
  data[data['contcod'] == country][['x', 'y']].apply(lambda row: append_to_list(row['x'], row['y']), axis=1)
    
  dict_for_export[country] = temp_list

Preview contents:

In [47]:
dict_for_export['BRA'][:3]

[{'x': 2.5, 'y': 13.46}, {'x': 7.5, 'y': 32.6}, {'x': 12.5, 'y': 42.1}]

In [48]:
dict_for_export['DEU'][17:20]

[{'x': 87.5, 'y': 97.45}, {'x': 92.5, 'y': 98.34}, {'x': 97.5, 'y': 99.54}]

Save to file:

In [49]:
import json

In [50]:
output = json.dumps(dict_for_export)

In [51]:
with open("./export/DATA_2011.js", "w") as text_file:
    text_file.write("var DATA = " + output)

# Create .js file with countries sorted by median

Now let's sort countries by their *median* so they show up in that order on the website chart in the legend

And export as `ALL_COUNTRIES_CODES` for consumption in JS

In [52]:
medians = data.groupby('contcod')['y'].median()

In [53]:
medians.head(5)

contcod
ALB    61.960
ARG    78.775
ARM    34.465
AUS    91.175
AUT    95.060
Name: y, dtype: float64

In [54]:
type(medians)

pandas.core.series.Series

In [55]:
sorted = medians.sort_values()

In [56]:
sorted.head(5)

contcod
MDG     2.380
RWA     3.295
MWI    10.425
LSO    11.175
GNB    13.405
Name: y, dtype: float64

In [57]:
sorted.tail(5)

contcod
DNK    93.605
LTU    94.480
NOR    94.735
AUT    95.060
LUX    95.785
Name: y, dtype: float64

In [58]:
all_countries = list(sorted.index)

In [59]:
all_countries[:5]

['MDG', 'RWA', 'MWI', 'LSO', 'GNB']

In [60]:
all_countries[-5:]

['DNK', 'LTU', 'NOR', 'AUT', 'LUX']

In [63]:
len(all_countries)

107

Note -- countries are in *increasing* order at the moment, so we `reverse()`

In [65]:
all_countries.reverse()

In [66]:
all_countries[:5]

['LUX', 'AUT', 'NOR', 'LTU', 'DNK']

In [67]:
all_countries_string = json.dumps(all_countries)

In [68]:
all_countries_string[:24]

'["LUX", "AUT", "NOR", "L'

In [69]:
with open("./export/DATA_2011_COUNTRIES.js", "w") as text_file:
    text_file.write("var ALL_COUNTRIES_CODES = " + all_countries_string)

# Generate .js list of countries in alphabetical order

note that we can't just alphabetize the ISO3 strings as they don't always correlate to alphabetical order, e.g. `GBR` is `United Kingdom`

## Build lookup dictionary

In [70]:
ISO = pd.read_csv('./countries/iso2_iso3.csv')

In [71]:
ISO.head(3)

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""


In [72]:
ISO.columns = ['country', 'iso2', 'iso3', 'u0', 'u1', 'u2']

In [73]:
countries = ISO[['country', 'iso2', 'iso3']]

In [74]:
countries.head(3)

,country,iso2,iso3
0,Afghanistan,"""AF""","""AFG"""
1,Albania,"""AL""","""ALB"""
2,Algeria,"""DZ""","""DZA"""


In [75]:
countries['iso3'] = countries['iso3'].apply(lambda x: x.replace('"', ''))
countries['iso3'] = countries['iso3'].apply(lambda x: x.replace(' ', ''))
countries['iso2'] = countries['iso2'].apply(lambda x: x.replace('"', ''))
countries['iso2'] = countries['iso2'].apply(lambda x: x.replace(' ', ''))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [76]:
countries.head(3)

,country,iso2,iso3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA


# Add South Sudan

South Sudan was added in 2011 but its data is not on the `iso2_iso3.csv`

We will need this country in our lookup dictionary, as we have the PPP data for it.

In [77]:
sudan = { 'country': 'South Sudan', 'iso2': 'SS', 'iso3': 'SSD'}

In [78]:
countries = countries.append(sudan, ignore_index=True)

In [79]:
countries.tail(3)

,country,iso2,iso3
254,Zambia,ZM,ZMB
255,Zimbabwe,ZW,ZWE
256,South Sudan,SS,SSD


In [80]:
iso3_to_name = pd.Series(countries['country'].values, index=countries['iso3']).to_dict()

## Use Dict to create list

In [81]:
all_countries[:10]

['LUX', 'AUT', 'NOR', 'LTU', 'DNK', 'USA', 'CAN', 'NLD', 'FIN', 'DEU']

In [82]:
df_order = pd.DataFrame(all_countries, columns=['iso3'])

In [83]:
df_order.head(4)

,iso3
0,LUX
1,AUT
2,NOR
3,LTU


In [84]:
iso3_to_name.get('LUX')

'Luxembourg'

In [85]:
df_order['name'] = df_order.apply(lambda x: iso3_to_name.get(x['iso3']), axis=1)

In [86]:
df_order.head(4)

,iso3,name
0,LUX,Luxembourg
1,AUT,Austria
2,NOR,Norway
3,LTU,Lithuania


In [87]:
my_sorted_list = df_order.sort_values(['name'])['iso3']

In [88]:
final_list_sorted = list(my_sorted_list)

In [89]:
final_list_sorted[:10]

['ALB', 'ARG', 'ARM', 'AUS', 'AUT', 'BGD', 'BEN', 'BTN', 'BOL', 'BWA']

In [90]:
countries_sorted_string = json.dumps(final_list_sorted)

In [91]:
with open("./export/DATA_2011_COUNTRIES_SORTED.js", "w") as text_file:
    text_file.write("var ALL_COUNTRIES_CODES_SORTED = " + countries_sorted_string)

# LIST OF ISO2 NAMES OF COUNTRIES NOT IN DATASET 

We'll need to have a list of countries not in the dataset so we can color the countries differently on the map

In [92]:
countries_on_map = ["AE","AF","AL","AM","AO","AR","AT","AU","AZ","BA","BD","BE","BF","BG","BI","BJ","BN","BO","BR","BS","BT","BW","BY","BZ","CA","CD","CF","CG","CH","CI","CL","CM","CN","CO","CR","CU","CY","CZ","DE","DJ","DK","DO","DZ","EC","EE","EG","EH","ER","ES","ET","FI","FJ","FK","FO","FR","GA","GB","GE","GH","GL","GM","GN","GQ","GR","GT","GW","GY","HN","HR","HT","HU","ID","IE","IL","IN","IQ","IR","IS","IT","JM","JO","JP","KE","KG","KH","KM","KP","KR","KW","KZ","LA","LB","LK","LR","LS","LT","LU","LV","LY","MA","MD","ME","MG","MK","ML","MM","MN","MR","MW","MX","MY","MZ","NA","NC","NE","NG","NI","NL","NO","NP","NZ","OM","PA","PE","PF","PG","PH","PK","PL","PR","PS","PT","PY","QA","RO","RS","RU","RW","SA","SB","SD","SE","SI","SK","SL","SN","SO","SR","SS","SV","SY","SZ","TD","TG","TH","TJ","TL","TM","TN","TR","TT","TW","TZ","UA","UG","US","UY","UZ","VE","VN","VU","WS","YE","ZA","ZM","ZW"]

In [93]:
countries_on_map[:10]

['AE', 'AF', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA']

In [94]:
countries_in_dataset = list(data['contcod'].unique())

All countries in the dataset are here:

In [95]:
len(all_countries)

107

Lookup dictionary:

In [96]:
iso2_to_iso3 = pd.Series(countries['iso3'].values, index=countries['iso2']).to_dict()

In [97]:
not_in_dataset = []

for iso2 in countries_on_map:
  iso3 = iso2_to_iso3.get(iso2)
  if iso3 not in all_countries:
    not_in_dataset.append(iso2)

In [98]:
len(not_in_dataset)

73

### Write a file for JS consumption

In [99]:
import json

In [100]:
output = json.dumps(not_in_dataset)

In [101]:
with open("./countries/NOT_IN_DATASET.js", "w") as text_file:
    text_file.write("var COUNTRIES_NOT_IN_DATASET = " + output)

***

## Misc exploration

See how many countries ever dip below 5th percentlie

In [102]:
data[data['y'] < 5]['contcod'].unique().size

27

Which countries last ventile is above 99.5 income percentile:

In [103]:
data[data['y'] > 99.5]['contcod'].unique()

array(['AUS', 'AUT', 'CAN', 'DEU', 'DNK', 'FRA', 'GBR', 'LTU', 'LUX',
       'NOR', 'NZL', 'USA'], dtype=object)